In [9]:
# Import keras and other Utils
import pltutils as pltu
import matplotlib.pyplot as plt
import numpy as np
import math
import cv2
import tensorflow as tf
import os


from keras.datasets import mnist
from keras.datasets import cifar10
from keras.utils import np_utils
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Conv2D
from keras.layers import Reshape, MaxPooling2D, Flatten
from keras.layers import Input, InputLayer, Lambda, Dropout
from keras.layers import BatchNormalization,Softmax
from keras.optimizers import RMSprop
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator 
from keras.callbacks import ModelCheckpoint

In [10]:
%matplotlib inline
plt.style.use('seaborn-whitegrid')

In [11]:
# Create Model
model = Sequential()
model.add(Conv2D(filters=64,kernel_size=(3,3),strides=(1,1),padding='same',input_shape=(224,224,3),activation='relu'))
model.add(Conv2D(filters=64,kernel_size=(3,3),strides=(1,1),padding='same',activation='relu'))

model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='valid'))
model.add(Conv2D(128,(3,3),padding='same',activation='relu'))
model.add(Conv2D(128,(3,3),padding='same',activation='relu'))

model.add(MaxPooling2D((2,2)))
model.add(Conv2D(256,(3,3),padding='same',activation='relu'))
model.add(Conv2D(256,(3,3),padding='same',activation='relu'))
model.add(Conv2D(256,(3,3),padding='same',activation='relu'))

model.add(MaxPooling2D((2,2)))
model.add(Conv2D(512,(3,3),padding='same',activation='relu'))
model.add(Conv2D(512,(3,3),padding='same',activation='relu'))
model.add(Conv2D(512,(3,3),padding='same',activation='relu'))

model.add(MaxPooling2D((2,2)))
model.add(Conv2D(512,(3,3),padding='same',activation='relu'))
model.add(Conv2D(512,(3,3),padding='same',activation='relu'))
model.add(Conv2D(512,(3,3),padding='same',activation='relu'))

model.add(MaxPooling2D((2,2)))
model.add(Flatten())
model.add(Dense(4096,activation='relu'))
model.add(Dense(4096,activation='relu'))
model.add(Dense(1000,activation='softmax'))
print (model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 56, 56, 128)       0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 56, 56, 256)      

In [12]:
import keras
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer = keras.optimizers.SGD(lr=0.005, decay=1e-6, momentum=0.9, nesterov=True),
              metrics=['accuracy'])

In [13]:

batch_size = 32
 

#实例化对象datagen
datagen=ImageDataGenerator()   
 
#读训练集图片
train_generator = datagen.flow_from_directory(
    'e:/tensorflow/data/ImageNet/train',
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=batch_size)

validation_generator = datagen.flow_from_directory(
    'e:/tensorflow/data/ImageNet/val',
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=batch_size)
 

Found 1281167 images belonging to 1000 classes.
Found 50000 images belonging to 1000 classes.


In [6]:
import os
checkpoint_path = "checkpoints/vgg16-train1000/checkpoint-weights_only-epoch{epoch:04d}-val_accuracy{val_accuracy:.4f}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
print(checkpoint_dir)
cp_callback=ModelCheckpoint(checkpoint_path, verbose=1, save_weights_only=True,period=1)

checkpoints/vgg16-train1000


In [7]:
model.save('checkpoints/vgg16-train1000/model.h5')

In [7]:
model.load_weights('./checkpoints/vgg16-train1000/checkpoint-weights_only-epoch0002-val_accuracy0.2490.ckpt')

In [8]:
model.fit_generator(generator=train_generator,steps_per_epoch=1281167/batch_size,epochs=1
                   ,callbacks=[cp_callback],
                   validation_data=validation_generator,validation_steps=batch_size)


Epoch 1/1
16546/40036 [===========>..................] - ETA: 2:53:50 - loss: 20.2156 - accuracy: 0.1320

C:\ProgramData\Anaconda3\envs\tf-gpu\lib\site-packages\PIL\TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


40037/40036 [==============================] - 17595s 439ms/step - loss: 12.4080 - accuracy: 0.0551 - val_loss: 6.9039 - val_accuracy: 0.0000e+00

Epoch 00001: saving model to checkpoints/vgg16-train1000/checkpoint-weights_only-epoch0001-val_accuracy0.0000.ckpt


In [10]:
model.load_weights('./checkpoints/vgg16-train1000/checkpoint-weights_only-epoch0001-accuracy0.1332.ckpt')
model.evaluate_generator(generator=validation_generator)

[2.780583381652832, 0.26194000244140625]

In [12]:
model.evaluate_generator(generator=validation_generator)

[4.218497276306152, 0.26109999418258667]